# Necessary imports

In [4]:
import (
	"fmt"
	"image/color"
	"math"
	"bytes"
	"math/rand"

	"gonum.org/v1/gonum/mat"
	"gonum.org/v1/gonum/stat/distuv"
	"gonum.org/v1/plot"
	"gonum.org/v1/plot/plotter"
	"gonum.org/v1/plot/vg"
)

# Problem 1

In [42]:
func p1_f(x float64) float64 { return math.Pow(1.0 + x, -0.5) }
func p1_F(x float64) float64 { return 2.0*math.Pow(1.0 + x, 0.5)} // anti-derivative of f

func p1_trap(a float64, b float64, N int) float64 {

	// Step size (determined by # of points)
	h := (b-a) / float64(N)

	iterations := 0
	sum := 0.0
	for x_i := h; x_i < 1.0; x_i += h {

		y_i := p1_f(x_i)
		sum += y_i
		iterations++
	}
	fmt.Printf("%d\n", iterations)

	endpoint_0 := p1_f(0.0) / 2.0
	endpoint_n := p1_f(1.0) / 2.0

	Q := h * (endpoint_0 + sum + endpoint_n)
	fmt.Printf("Q = %f\n", Q)

	return Q

}


In [51]:
func p1_monte(N int) []float64 {
	
	Y := 0.0
	areas := make([]float64, N)
	
	for i := 1; i <= N; i++ {
	
		X := rand.Float64()
		Y += p1_f(X)
	
		areas[i-1] = Y / float64(i)
	}
	
	fmt.Println(areas[N-1])
	return areas
}


In [67]:
// Calculate errors
N := 1000
exact_value := p1_F(1.0) - p1_F(0.0)
trapezoid_error := math.Abs(exact_value - p1_trap(0.0, 1.0, N))
monte_error := math.Abs(exact_value - p1_monte(N)[N-1])

fmt.Printf("Trapezoid error: %E\n", trapezoid_error)
fmt.Printf("Monte-carlo error: %E\n", monte_error)

999
Q = 0.828427
0.8291258755930432
Trapezoid error: 2.693527E-08
Monte-carlo error: 6.987508E-04


32 <nil>

# Problem  

We can use this formula to calculate the price of the option:

$E\left[\max \left(S_T-K, 0\right) \mid S 0=s\right] \approx \frac{1}{N} \sum_{n=1}^N \max \left(S_T[n]-K, 0\right)$

where 

$S_i(T)=e^{-\sigma^2 T / 2+\sigma W_i(T)} s_i$


In [69]:
// Code to create a generic stock S(T) with variance T
func W(T float64) float64 {return rand.NormFloat64() * math.Sqrt(T)}
func create_stock(T float64) func() float64 {
	return func() float64 {
		return s * math.Exp(- math.Pow(sigma, 2) * (T / 2.0) + sigma * W(T))
	}
}

// Calulate the payoff of on a call option
func payoff(stocks []func() float64) float64 {
	// Retrive largest stock
	var max_stock float64
	for _, stock := range stocks {
		max_stock = math.Max(max_stock, stock())
	}
	return math.Max(0, max_stock - K)
}

// Calculate the option price using Monte-Carlo
func monte_carlo_price(stocks []func() float64, N int) float64 {
	var sum float64
	for i := 0; i < N; i++ {
		sum += payoff(stocks)
	}
	return sum / float64(N)
}



In [70]:
T 				 := 1.0  // Variance
sigma 		 := 0.2  // Standard deviation
K 				 := 1.22 // Strike price
s 				 := K    // Condition
iterations := 1000000

// Calculate the option price using Black-Scholes
func black_scholes() float64 {
	d1 := (math.Log(s / K) + (math.Pow(sigma, 2) / 2.0) * T) / (sigma * math.Sqrt(T))
	d2 := d1 - sigma * math.Sqrt(T)
	dist := distuv.UnitNormal
	return s * dist.CDF(d1) - K * dist.CDF(d2)
}
b := black_scholes()

// Calculate the option price using Monte Carlo
stocks := make([]func() float64, 1)
stocks[0] = create_stock(T)
m := monte_carlo_price(stocks, iterations)

fmt.Printf("Black-Scholes: %f \t Monte Carlo: %f \t Error: %f\n", b, m, e)

Black-Scholes: 0.097180 	 Monte Carlo: 0.097049 	 Error: 0.000065


66 <nil>

# Problem 3

$E\left[\max \left(\max \left(S_1(T), S_2(T)\right)-K, 0\right), \mid S_1(0)=s_1, S_2(0)=s_2\right] \approx \frac{1}{N} \sum_{n=1}^N \max \left(\max \left(S_1[n], S_2[n]\right)-K, 0\right)$

In [73]:
T 				 := 1.0  // Variance
sigma 		 := 0.2  // Standard deviation
K 				 := 1.22 // Strike price
s 				 := K    // Condition
iterations := 10000

// Calculate the option price using Monte Carlo
stocks := make([]func() float64, 2)
stocks[0] = create_stock(T)
stocks[1] = create_stock(T)
m := monte_carlo_price(stocks, iterations)

fmt.Printf("Two stocks: %f\n", m)

Two stocks: 0.169842


21 <nil>

In [77]:
stock_count := 10

// Calculate option price for a specified number of stocks
stocks := make([]func() float64, stock_count)
for i := 0; i < stock_count; i++ {
	stocks[i] = create_stock(T)
}
m := monte_carlo_price(stocks, iterations)

fmt.Printf("%d stocks: %f\n", stock_count, m)

10 stocks: 0.418486


20 <nil>

# Problem 4

# Problem 5